# 1. Introduction

Diabetes is a chronic disease that occurs when the pancreas is no longer able to produce insulin or when the body cannot make full use of the insulin it produces. In this case, our body’s blood sugar will rise, which will lead to a chronic damage dysfunction of various tissues, especially the eyes, kidneys, heart, blood vessels, and nerves.

In this project, we will utilize machine learning algorithms to predict early-stage diabetes risk.

## 1.1 Imports

To keep our project clean, we have consolidated all our imports in the cell below.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import tree, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

# 2. Pre-processing and Data Cleaning

In [ ]:
diabetes = pd.read.csv("diabetes_data_upload.csv")

# 3. Exploratory Data Analysis

# 4. Model Selection

# 5. Modeling

# 6. Result

# 7. Conclusion

# 8. Limitation